In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [2]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical

Using TensorFlow backend.


In [3]:
batch_size = 512
num_classes = 10
epochs = 12

In [4]:
def load_mnist_data():
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train, x_test = x_train/255.0, x_test/255.0
    x_train, x_test = x_train.astype(np.float32), x_test.astype(np.float32)    
    x_train, x_test = np.expand_dims(x_train, 3), np.expand_dims(x_test, 3)
    y_test_int = y_test # keep labels in integer form
    y_train, y_test = to_categorical(y_train), to_categorical(y_test)
    return x_train, x_test, y_train, y_test, y_test_int

x_train, x_test, y_train, y_test, y_test_int = load_mnist_data()

In [5]:
# input image dimensions
img_rows, img_cols = 28, 28

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test= x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'validation samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 validation samples


In [6]:
# Use the built in data generation features of Keras
datagen = ImageDataGenerator(
    width_shift_range = 0.075,
    height_shift_range = 0.075,
    rotation_range = 12,
    shear_range = 0.075,
    zoom_range = 0.05,
    fill_mode = 'constant',
    cval = 0
)
datagen.fit(x_train)

In [10]:
# Build the model
model = Sequential()
model.add(Conv2D(32, kernel_size = (5, 5),
                 activation = 'relu',
                 input_shape = input_shape))
model.add(Conv2D(32, kernel_size = (3, 3),
                 activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation = 'relu'))
model.add(Conv2D(64, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(1024, activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(num_classes, activation = 'softmax'))


model.compile(loss = keras.losses.categorical_crossentropy,
              optimizer = keras.optimizers.SGD(lr=0.01),
              metrics = ['accuracy'])


filepath='models/mnist_conv_deep_slowlearning{epoch:02d}.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=False)


model.fit_generator(datagen.flow(x_train,
                                  y_train,
                                  batch_size = batch_size),
                    epochs = epochs,
                    steps_per_epoch = x_train.shape[0]/32,
                    verbose = 1,
                    validation_data = (x_test, y_test),
                    callbacks = [checkpoint])

score = model.evaluate(x_test, y_test, verbose = 0)
print('Validation loss:', score[0])
print('Validation accuracy:', score[1])


Epoch 1/12
  31/1875 [..............................] - ETA: 52:58 - loss: 2.3046 - acc: 0.0992

KeyboardInterrupt: 